# 高度なディープラーニングのベストプラクティス

- Keras Functional API    
- Keras のコールバックの使用    
- TensorBoardの操作    
- 最先端のモデルを開発するための重要なベストプラクティス    

# 多入力モデル

In [0]:
from keras.models import Model
from keras import layers
from keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

text_input = Input(shape=(None,), dtype='int32', name='text')

embedded_text = layers.Embedding(text_vocabulary_size, 64)(text_input)
encoded_text = layers.LSTM(32)(embedded_text)

question_input = Input(shape=(None,), dtype='int32', name='question')
embedded_question = layers.Embedding(question_vocabulary_size, 32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

concatenated = layers.concatenate([encoded_text, encoded_question], axis=-1)

answer = layers.Dense(answer_vocabulary_size, activation='softmax')(concatenated)

model = Model([text_input, question_input], answer)
model.compile(optimizer='rmsprop',
             loss='categorical_crossentropy',
             metrics=['acc'])

In [10]:
#多入力モデルへのデータの供給
import numpy as np
num_samples = 1000
max_length = 100

text = np.random.randint(1, text_vocabulary_size,
                        size=(num_samples, max_length))
question = np.random.randint(1, question_vocabulary_size,
                            size=(num_samples, max_length))

answers = np.zeros(shape=(num_samples, answer_vocabulary_size))
indices = np.random.randint(0, answer_vocabulary_size, size=num_samples)
for i, x in enumerate(answers):
    x[indices[i]] = 1

#入力リストを使った適応
model.fit([text, question], answers, epochs=10, batch_size=128)



Epoch 1/10
1000/1000 [==============================] - 5s 5ms/step - loss: 6.2145 - acc: 0.0020
Epoch 2/10
1000/1000 [==============================] - 3s 3ms/step - loss: 6.1982 - acc: 0.0300
Epoch 3/10
1000/1000 [==============================] - 3s 3ms/step - loss: 6.1633 - acc: 0.0070
Epoch 4/10
1000/1000 [==============================] - 3s 3ms/step - loss: 6.0816 - acc: 0.0110
Epoch 5/10
1000/1000 [==============================] - 3s 3ms/step - loss: 6.0053 - acc: 0.0080
Epoch 6/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.9220 - acc: 0.0110
Epoch 7/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.8593 - acc: 0.0140
Epoch 8/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.8000 - acc: 0.0140
Epoch 9/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.7283 - acc: 0.0200
Epoch 10/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.6574 - acc: 0.0220


In [11]:
#入力ディクショナリを使用
model.fit({'text': text, 'question': question}, answers, epochs=10, batch_size=128)


Epoch 1/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.5801 - acc: 0.0320
Epoch 2/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.5145 - acc: 0.0390
Epoch 3/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.4371 - acc: 0.0490
Epoch 4/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.3680 - acc: 0.0560
Epoch 5/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.3023 - acc: 0.0710
Epoch 6/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.2469 - acc: 0.0770
Epoch 7/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.1673 - acc: 0.0800
Epoch 8/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.1098 - acc: 0.0850
Epoch 9/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.0571 - acc: 0.0860
Epoch 10/10
1000/1000 [==============================] - 3s 3ms/step - loss: 4.9794 - acc: 0.1040


# 多出力モデル

In [0]:
from keras import layers
from keras import Input
from keras.models import Model

vocabulary_size = 50000
num_income_groups = 10

posts_input = Input(shape=(None,) ,dtype='int32', name='posts')

embedded_posts = layers.Embedding(256, vocabulary_size)(posts_input)
x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

age_prediction = layers.Dense(1, name='age')(x)
income_prediction = layers.Dense(num_income_groups,
                                activation='softmax',
                                name='income')(x)
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)
model = Model(posts_input,
             [age_prediction, income_prediction, gender_prediction])

In [0]:
model.compile(optimizer='rmsprop',
             loss=['mse',
                  'categorical_crossentropy',
                  'binary_crossentropy'])

In [0]:
#多入力モデルへのデータの供給
import numpy as np
num_samples = 1000

posts = np.random.randint(1, size=(num_samples, 1))
age_targets = np.random.randint(1, size=(num_samples, 1))
income_targets = np.random.randint(1, size=(num_samples, 10))
gender_targets = np.random.randint(1, size=(num_samples, 1))


In [29]:
model.fit(posts, [age_targets, income_targets, gender_targets], epochs=1, batch_size=128)


Epoch 1/1


ResourceExhaustedError: ignored

In [0]:
model.compile(optimizer='rmsprop',
             loss={'age': 'mse',
                   'income': 'categorical_crossentropy',
                   'gender': 'binary_crossentropy'
             })

損失関数の貢献度が不均衝(ふきんこう)である場合はモデルの表現は最も大きい損失値を持つタスクを優先する形で最適化される。    

この問題を解決するには損失に重要度を割り当てる。

In [0]:
model.compile(optimizer='rmsprop',
             loss=['mse',
                  'categorical_crossentropy',
                  'binary_crossentropy'],
             loss_weights=[0.25, 1., 10.])

In [0]:
model.compile(optimizer='rmsprop',
             loss={'age': 'mse',
                   'income': 'categorical_crossentropy',
                   'gender': 'binary_crossentropy'},
             loss_weights={'age':0.25 ,'income': 1.,'gender': 10.})

In [0]:
#入力リストを使った適応
model.fit(posts, [age_targets, income_targets, gender_targets], epochs=10, batch_size=128)
